In [1]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import Document
import json
import os



c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load INSTRUCTOR_Transformer
max_seq_length  512


c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

In [ ]:

def load_json_data(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            text = f"title: {data['title']}\nabstract: {data['abstract']}"
            doc = Document(page_content=text)
            documents.append(doc)
    return documents


In [ ]:
def vector_database(data_location, file_location):
    documents = load_json_data(data_location)
    embeddings = HuggingFaceInstructEmbeddings()
    vectordb = FAISS.from_documents(documents, embedding=embeddings)
    vectordb.save_local(file_location)
    return embeddings

def get_llm():
    GOOGLE_API_KEY = '_________________'  # Set your API key here
    llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.1)
    return llm


In [ ]:
def get_prompt():
    prompt_template = """
    Given the following context and a question, generate an answer based on the context only.
    In the answer try to provide as much text as possible from the "response" section in the source document.

    CONTEXT: {context}

    QUESTION: {question}
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

In [ ]:
def get_chain(file_location):
    llm = get_llm()
    embeddings = HuggingFaceInstructEmbeddings()
    vectordb = FAISS.load_local(file_location, embeddings, allow_dangerous_deserialization=True)
    retriever = vectordb.as_retriever()
    prompt = get_prompt()
    
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        input_key="question", 
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt} 
    )
    
    return chain

In [2]:
file_location="vectorDB/"
chain = get_chain(file_location)

query = "What is the World as Evolving Information.?"
query=query+"Summarise it in 50 words"
try:
    response = chain({"question": query})
    print(response)
except Exception as e:
    print("Error:", e)

load INSTRUCTOR_Transformer


c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

max_seq_length  512


c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'question': 'What is the World as Evolving Information.?Summarise it in 50 words', 'result': 'This paper discusses the benefits of describing the world as information, especially in the study of the evolution of life and cognition. Traditional studies encounter problems because it is difficult to describe life and cognition in terms of matter and energy, since their laws are valid only at the physical scale. However, if matter and energy, as well as life and cognition, are described in terms of information, evolution can be described consistently as information becoming more complex.', 'source_documents': [Document(page_content='title: The World as Evolving Information\nabstract:   This paper discusses the benefits of describing the world as information,\nespecially in the study of the evolution of life and cognition. Traditional\nstudies encounter problems because it is difficult to describe life and\ncognition in terms of matter and energy, since their laws are valid only at the\nph

In [3]:
information=response['source_documents']
information

[Document(page_content='title: The World as Evolving Information\nabstract:   This paper discusses the benefits of describing the world as information,\nespecially in the study of the evolution of life and cognition. Traditional\nstudies encounter problems because it is difficult to describe life and\ncognition in terms of matter and energy, since their laws are valid only at the\nphysical scale. However, if matter and energy, as well as life and cognition,\nare described in terms of information, evolution can be described consistently\nas information becoming more complex.\n  The paper presents eight tentative laws of information, valid at multiple\nscales, which are generalizations of Darwinian, cybernetic, thermodynamic,\npsychological, philosophical, and complexity principles. These are further used\nto discuss the notions of life, cognition and their evolution.\n'),
 Document(page_content='title: Ensemble Learning for Free with Evolutionary Algorithms ?\nabstract:   Evolutionary L

In [4]:
summarization=response['result']
summarization

'This paper discusses the benefits of describing the world as information, especially in the study of the evolution of life and cognition. Traditional studies encounter problems because it is difficult to describe life and cognition in terms of matter and energy, since their laws are valid only at the physical scale. However, if matter and energy, as well as life and cognition, are described in terms of information, evolution can be described consistently as information becoming more complex.'

In [5]:
file_location="vectorDB/"
chain = get_chain(file_location)

query = "What is 'The World as Evolving Information'.?"
query=query+"Explain it"
try:
    response = chain({"question": query})
    print(response['result'])
except Exception as e:
    print("Error:", e)

load INSTRUCTOR_Transformer


c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\91975\anaconda3\envs\langchain\lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

max_seq_length  512
This paper discusses the benefits of describing the world as information, especially in the study of the evolution of life and cognition. Traditional studies encounter problems because it is difficult to describe life and cognition in terms of matter and energy, since their laws are valid only at the physical scale. However, if matter and energy, as well as life and cognition, are described in terms of information, evolution can be described consistently as information becoming more complex.


In [19]:


def info(query):
    try:
        query=query+"Give all information about it"
        response = chain({"question": query})
        return response['result']
    except Exception as e:
        print("Error:", e)

def summary(query):
    query=query+"Summarise it in 50 words"
    try:
        response = chain({"question": query})
        return response['result']
    except Exception as e:
        print("Error:", e)

def explaination(query):
    query=query+"Explain it in 200 words"
    try:
        response = chain({"question": query})
        return response['result']
    except Exception as e:
        print("Error:", e)

In [20]:
query = "What is the World as Evolving Information.?"
print(info(query))


This paper discusses the benefits of describing the world as information, especially in the study of the evolution of life and cognition. Traditional studies encounter problems because it is difficult to describe life and cognition in terms of matter and energy, since their laws are valid only at the physical scale. However, if matter and energy, as well as life and cognition, are described in terms of information, evolution can be described consistently as information becoming more complex. The paper presents eight tentative laws of information, valid at multiple scales, which are generalizations of Darwinian, cybernetic, thermodynamic, psychological, philosophical, and complexity principles. These are further used to discuss the notions of life, cognition and their evolution.


In [23]:
query = "What is the World as Evolving Information.?"
print(summary(query))


This paper discusses the benefits of describing the world as information, especially in the study of the evolution of life and cognition. Traditional studies encounter problems because it is difficult to describe life and cognition in terms of matter and energy, since their laws are valid only at the physical scale. However, if matter and energy, as well as life and cognition, are described in terms of information, evolution can be described consistently as information becoming more complex.


In [22]:
query = "What is the World as Evolving Information.?"
print(explaination(query))


The World as Evolving Information is a theory that proposes describing the world in terms of information, rather than matter and energy. This approach is particularly useful in studying the evolution of life and cognition, as the laws of matter and energy are not sufficient to explain these phenomena.

By describing the world in terms of information, evolution can be consistently described as information becoming more complex. This theory is supported by eight tentative laws of information, which are generalizations of Darwinian, cybernetic, thermodynamic, psychological, philosophical, and complexity principles. These laws can be used to discuss the notions of life, cognition, and their evolution.

The World as Evolving Information theory has several advantages over traditional approaches. First, it provides a unified framework for describing the evolution of life and cognition. Second, it allows for a more nuanced understanding of the relationship between information and matter/energy